**Homework 25**

In this assignment your will train a RNN to predict characters of *Alice in Wonderland*, from strings of consecutive characters.

We begin as usual with the imports you will need for this assignment.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
device=('cuda' if torch.cuda.is_available()
        else 'cpu')

device

'cpu'

Run the following text block to read *Alice in Wonderland* from the web, store it in the variable `text`, convert to lower case and remove punctuation.

In [3]:
import string
from urllib.request import urlopen
url='https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt'
text = urlopen(url).read().decode('utf-8')
text=text.lower()
text=[c for c in text if (c not in string.punctuation) and (c!='\n')]

Write a class `Tokenizer` with the following methods:


*   `__init__`, a method that builds a dictionary `tokens` whose keys are the set of unique characters in some input `text`, and values are integers.
*   `encode`, a method that takes in a corpus of text, converts each character according to the dictionary built by the __init__ method, and outputs a list of those integers.
*   `decode`, a method that takes a single integer (a value from the dictionary), and returns the corresponding character key.



In [107]:
class Tokenizer():
  def __init__(self,text):
    self.tokens = {}
    for chr in text:
      if chr ==' ':
        self.tokens[chr] = 26
      elif chr == '3':
        self.tokens[chr] = 27
      elif chr == '0':
        self.tokens[chr] = 28
      else:
        self.tokens[chr] = ord(chr) - ord('a')

  def encode(self,text):
    return [self.tokens[chr] for chr in text]

  def decode(self,n):
    for key in self.tokens:
      if self.tokens[key] == n:
        return key

Now, create an object called `tok` of your `Tokenizer` class, and use it to encode `text` as a list of integers, `text_indices`.

In [161]:
tok= Tokenizer(text)
text_indices= tok.encode(text)
list(zip(text_indices, text))

[(0, 'a'),
 (11, 'l'),
 (8, 'i'),
 (2, 'c'),
 (4, 'e'),
 (18, 's'),
 (26, ' '),
 (0, 'a'),
 (3, 'd'),
 (21, 'v'),
 (4, 'e'),
 (13, 'n'),
 (19, 't'),
 (20, 'u'),
 (17, 'r'),
 (4, 'e'),
 (18, 's'),
 (26, ' '),
 (8, 'i'),
 (13, 'n'),
 (26, ' '),
 (22, 'w'),
 (14, 'o'),
 (13, 'n'),
 (3, 'd'),
 (4, 'e'),
 (17, 'r'),
 (11, 'l'),
 (0, 'a'),
 (13, 'n'),
 (3, 'd'),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (0, 'a'),
 (11, 'l'),
 (8, 'i'),
 (2, 'c'),
 (4, 'e'),
 (18, 's'),
 (26, ' '),
 (0, 'a'),
 (3, 'd'),
 (21, 'v'),
 (4, 'e'),
 (13, 'n'),
 (19, 't'),
 (20, 'u'),
 (17, 'r'),
 (4, 'e'),
 (18, 's'),
 (26, ' '),
 (8, 'i'),
 (13, 'n'),
 (26, ' '),
 (22, 'w'),
 (14, 'o'),
 (13, 'n'),
 (3, 'd'),
 (4, 'e'),
 (17, 'r'),
 (11, 'l'),
 (0, 'a'),
 (13, 'n'),
 (3, 'd'),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, ' '),
 (26, 

For convenience, we'll define `vocab_size=len(tok.tokens)` to be the length of your tokenizer dictionary:

In [158]:
vocab_size=len(tok.tokens)
vocab_size

29

The next task is to create feature sequences and targets. From `text_indices`, create a list-of-lists `X`. Each sublist of `X` should correspond to 50 consecutive elements of `text_indices`. At the same time, create a list `y` which contains the indices of the characters that follow each sublist of `X`. For example, `X[0]` should be a list containing the first 50 elements of `text_indices`: `text_indices[0]` through `text_indices[49]`. `y[0]` should be the 51st element, `text_indices[50]`.

To keep the size of the feature and target vectors manageable, consecutive lists in `X` should be shifted by 3, so the overlap is 47 elements. Hence, `X[1]` should be a list containing the integers `text_indices[3]` through `text_indices[52]`, and `y[1]` should be the integer `text_indices[53]`.

In [162]:
seq_len=50
X=[]
y=[]
for i in range(0,len(text_indices)-seq_len-1,3):
    X += [text_indices[i:i+50]]
    y += [text_indices[i+51]]

# for i in range(0,len(text)-seq_len-1,3):
#     # X += [text[i:i+50]]
#     # print(text[i:i+50])
#     line = [tok.tokens.get(chr) for chr in text[i:i+50]]
#     X += [line]
#     y.append([tok.tokens[text[i+51]]])

In [167]:
# len(X)
# y

Convert `X` and `y` to torch tensors with the same names, and check their shapes. If done correctly, the shape of `X` should be (45539, 50) and the shape of `y` should be (45539, ):

In [168]:
X= torch.tensor(X)
y= torch.tensor(y)
X.shape, y.shape

/var/folders/2g/2z0h9cgx58gd67svd7wwq59w0000gn/T/ipykernel_59637/1137368055.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X= torch.tensor(X)
/var/folders/2g/2z0h9cgx58gd67svd7wwq59w0000gn/T/ipykernel_59637/1137368055.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y)


(torch.Size([45539, 50]), torch.Size([45539]))

Convert `X` to a one-hot encoded vector `OneHotX` of 0's and 1's, and check its shape. You should now have shape (45539,50,29). In other words, the vector `OneHotX` now contains 45,539 sequences of length 50, and each element of each sequence is a 29-dimensional vector of 28 zeros and a single one in the entry corresponding to some character in the text.

In [192]:
OneHotX= F.one_hot(X, vocab_size).float()
OneHotX.shape

torch.Size([45539, 50, 29])

You're now ready to create your model, which will consist of two seperate one-layer pytorch models. The first will be a recurrent layer that takes in sequences of 29-dimensional vectors, and has a 128 dimensional hidden state. The second will ve a linear layer that will take the last hidden state and produce a 29 dimensional vector.

In [198]:
rnn=nn.RNN(29, 128, batch_first=True)
fc=nn.Linear(128, 29)

Compile your model using the `Adam` optimizer and an approporiately chosen loss function.

In [199]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(rnn.parameters()) + list(fc.parameters()), lr=0.001)

In [205]:
fc(rnn(batch_X)[1].squeeze()).shape

torch.Size([32, 29])

In [204]:
batch_y.shape

torch.Size([32])

Fit your data to X and y. Train for 50 epochs with a batch size of 128. Each epoch will take about 95 seconds, so you'll want to leave your computer for about an hour for this to complete.

In [207]:
n_epochs=50
N = OneHotX.shape[0]  # total number of observations in training data
batch_size=32

rnn.train()
for epoch in range(n_epochs):
  epoch_loss = 0.0

  # Shuffle the indices
  indices = torch.randperm(N,device=device)

  # Create mini-batches
  for i in range(0, N, batch_size):
    batch_indices = indices[i:i+batch_size]
    batch_X = OneHotX[batch_indices]
    batch_y = y[batch_indices]

    optimizer.zero_grad()

    rnn_result = rnn(batch_X)[1].squeeze()

    
    fc_result = fc(rnn_result)

    loss = criterion(fc_result, batch_y)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()*batch_size

  if epoch%2==0:
      avg_loss = epoch_loss / len(y)
      print(f"epoch: {epoch}, avg_loss: {avg_loss}")

epoch: 0, avg_loss: 2.526498697343249
epoch: 2, avg_loss: 2.4251642677214815


KeyboardInterrupt: 

We will now use your trained model to generate text, one character at a time. Run the following code block to do this. (It will take a minute or two to complete.) Its interesting that although the model generates one character at a time, you'll see very word-like strings in the final text.

In [ ]:
rnn.eval()
next_seq=OneHotX[:1]  #Initial "seed" sequence

newtext=''
with torch.no_grad():
  for i in range(500):
    seq=next_seq
    pred=fc(rnn(seq)[1].squeeze()) #predictions of your model
    pred_probs=torch.softmax(pred,dim=0).detach().cpu().numpy() #predictions->probs
    index_pred=np.random.choice(vocab_size,1,p=pred_probs)[0] #choose one
    newtext+=tok.decode(index_pred) #corresponding character

    next_vec=torch.zeros(vocab_size).to(device)
    next_vec[index_pred]=1  #one-hot encode chosen letter index
    next_seq=torch.zeros(1,seq_len,29).to(device)
    next_seq[0,:seq_len-1]=seq[0,1:] #new sequence is last 49 of old sequence
    next_seq[0,seq_len-1]=next_vec  #plus new vector

newtext #display generated text

**COPY AND PASTE THIS TEXT INTO THE SUBMISSION WINDOW ON GRADESCOPE**